In [4]:
import pandas as pd
import numpy as np
import re
import os
import glob

### 香蕉 => 暫以台中交易資料+南投中寮天氣資料做匹配

全國最大交易市場: 台中市場<BR>
南投主要生產區:南投縣中寮鄉
    
- 挑選最適農測站
- 挑選出需求資料
  - 欄位 => 氣壓、氣溫、露點溫度
  - 年份 => 先以2018以後資料為主，年份越早，資料越不完整

In [2]:
# 先嘗試處理2020氣溫資料(凍頂茶改??)

In [3]:
df = pd.read_csv("../data/Agricultural_Weather_Datas/凍頂茶改/2020平均氣溫(℃).csv", header=None).T
df.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,2020年 日/月,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,平均
1,1月,15.6,16.3,17.0,17.1,17.3,17.2,17.9,17.6,17.5,...,20.1,20.0,18.9,16.4,14.4,12.5,11.1,9.5,10.4,16
2,2月,12.4,14.6,15.4,14.8,15.6,16.3,16.5,14.9,13.7,...,17.9,18.4,20.4,20.8,18.9,19.2,20.8,NaN,NaN,16.7
3,3月,21.1,18.4,19.3,19.1,14.7,17.7,20.5,20.0,22.8,...,21.4,21.2,22.0,23.4,21.6,19.2,17.9,20.3,21.4,19.7
4,4月,20.9,18.9,19.5,18.9,16.7,17.3,17.8,18.7,19.8,...,18.3,16.9,19.8,21.3,21.7,22.6,22.5,23.0,NaN,19.9


In [4]:
df.drop(columns=[0,32], axis=1, inplace=True)
df.drop(index=0, inplace=True)
df

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
1,15.6,16.3,17.0,17.1,17.3,17.2,17.9,17.6,17.5,17.3,...,18.9,20.1,20.0,18.9,16.4,14.4,12.5,11.1,9.5,10.4
2,12.4,14.6,15.4,14.8,15.6,16.3,16.5,14.9,13.7,15.6,...,17.1,17.9,18.4,20.4,20.8,18.9,19.2,20.8,NaN,NaN
3,21.1,18.4,19.3,19.1,14.7,17.7,20.5,20.0,22.8,18.5,...,22.0,21.4,21.2,22.0,23.4,21.6,19.2,17.9,20.3,21.4
4,20.9,18.9,19.5,18.9,16.7,17.3,17.8,18.7,19.8,21.0,...,20.4,18.3,16.9,19.8,21.3,21.7,22.6,22.5,23.0,NaN
5,23.7,24.2,24.4,25.3,25.8,25.8,26.3,26.5,26.2,26.7,...,23.4,22.9,24.7,24.4,24.0,23.4,23.5,23.3,24.7,26.1
6,26.3,25.7,25.8,26.5,26.5,25.2,24.5,23.4,24.1,25.8,...,26.5,27.0,27.0,27.3,26.4,25.9,26.0,26.9,27.2,NaN
7,26.6,25.3,24.9,25.9,25.8,26.3,27.4,26.5,26.6,27.1,...,25.6,25.1,26.7,26.2,25.5,25.6,24.7,24.9,24.8,26.8
8,26.2,25.8,24.0,24.8,25.4,27.3,26.8,26.4,26.7,25.4,...,25.5,25.4,26.0,26.9,25.0,24.5,25.5,25.9,25.9,25.4
9,26.0,26.4,25.0,26.1,25.5,24.3,24.9,25.2,25.9,24.8,...,25.4,25.4,25.2,23.9,24.3,24.0,23.7,25.6,23.8,NaN
10,24.7,25.1,25.3,25.6,24.5,24.0,22.7,22.3,23.5,23.9,...,21.9,19.7,21.4,22.9,23.7,23.7,24.0,23.6,22.2,21.1


In [5]:
df.iloc[0,0]

'15.6'

### method1
依照資料順序與2020每日做結合
- 若有缺值出現在1~30天 => 如何處理
- 2月份特別處理 => 閏年與否

In [6]:
output = []

# 全年無缺值情況下
for col in df.columns:
    data = df[col].dropna().tolist()
    output += data
print(len(output))

366


In [7]:
time_2020 = pd.date_range("2020-01-01", "2020-12-31")

In [8]:
df_2020_tmp = pd.DataFrame({"日期":time_2020,
                           "最高溫度(℃)":output})
df_2020_tmp

,日期,最高溫度(℃)
0,2020-01-01,15.6
1,2020-01-02,12.4
2,2020-01-03,21.1
3,2020-01-04,20.9
4,2020-01-05,23.7
...,...,...
361,2020-12-27,26.1
362,2020-12-28,26.8
363,2020-12-29,25.4
364,2020-12-30,21.1


### mthod 2
日期(2020-month-day)以df.iloc[month, day]去匹配，如此一來可以確保每個值都完美匹配且避開缺值，但效率很差

In [9]:
time_2020 = pd.date_range("2020-01-01", "2020-12-31")

In [10]:
a = []
for day in time_2020:
    a.append(str(day))

In [11]:
test = list(map(lambda x: (int(x[5:7]), int(x[8:10])), a))

In [12]:
# for ele in test:
#     print(ele)
#     day = ele[1] - 1
#     mon = ele[0] - 1
#     print(df.iloc[day, mon])

In [13]:
whole_year_date = pd.date_range("2020-11-01", "2020-11-05")    
for day in whole_year_date:
    print((day.month, day.day))
    

(11, 1)
(11, 2)
(11, 3)
(11, 4)
(11, 5)


In [14]:
csv = "./data/Agricultural_Weather_Datas/凍頂茶改/2020平均地溫0cm(℃).csv"
print(csv)
# year = 2020
# re.findall(f"\\{year}(.*)\.csv", csv)



./data/Agricultural_Weather_Datas/凍頂茶改/2020平均地溫0cm(℃).csv


### 合併單一測站多份資料(延續上面的單份資料轉換，依據年份統合各欄位成一個dataframe)

將各測站每年資料統合成一個DATAFRAME，因為年份越久，資料完整度越差，因此先做2018~2020即可

In [16]:
df_time = pd.DataFrame({"日期":pd.date_range("2020-01-01", "2020-12-31")})
df_time

,日期
0,2020-01-01
1,2020-01-02
2,2020-01-03
3,2020-01-04
4,2020-01-05
...,...
361,2020-12-27
362,2020-12-28
363,2020-12-29
364,2020-12-30


# OLD

In [5]:
# 所有測站
for station in stations:
    path = f"./test_0711/{station}"
    if os.path.exists(path) != True:
        os.makedirs(path)
    
    # 只抓2019年
    for year in range(2019, 2020):
        whole_year = pd.date_range(f"{year}-01-01", f"{year}-12-31")
        df_output = pd.DataFrame({"日期": whole_year})
    
        for csv in glob.glob(f"../data/Agricultural_Weather_Datas/{station}/{year}*"):   # 抓出每個測站各年度資料
            try:
                df_col = csv_to_df(csv, year)
                df_output = pd.merge(df_output, df_col, on="日期", how="left")
         
            except:
                print(f"{station}-{year}-{file_name}: Something got wrong!")
             
        # 輸出為csv檔
        df_output.to_csv(f"{path}/{str(year)}_{station}.csv", index=False)
    
    print(f"{station}-{year}: Done" + "="*20)    

七股研究中心-2019: Done
凍頂茶改-2019: Done
口湖工作站-2019: Done
嘉義分場-2019: Done
嘉義農試-2019: Done
四湖植物園-2019: Done
埔里分場-2019: Done
恆春工作站-2019: Done
恆春畜試-2019: Done
旗南農改-2019: Done
東港工作站-2019: Done
林試六龜中心-2019: Done
林試扇平站-2019: Done
林試畢祿溪站-2019: Done
溪口農場-2019: Done
畜試所-2019: Done
種苗繁殖-2019: Done
義竹分場-2019: Done
臺中農改-2019: Done
臺南蘭花園區-2019: Done
臺南農改-2019: Done
臺大內茅埔-2019: Done
臺大和社-2019: Done
臺大溪頭-2019: Done
臺大竹山-2019: Done
臺大雲林校區-2019: Done
臺西水試所-2019: Done
蓮華池-2019: Done
農業試驗所-2019: Done
雲林分場-2019: Done
高雄農改-2019: Done
魚池茶改-2019: Done
鳳山農試-2019: Done


# Final Result

In [3]:
# 測站
stations = [i.split("\\")[1] for i in glob.glob("../data/Agricultural_Weather_Datas/*")]
stations

['七股研究中心',
 '凍頂茶改',
 '口湖工作站',
 '嘉義分場',
 '嘉義農試',
 '四湖植物園',
 '埔里分場',
 '恆春工作站',
 '恆春畜試',
 '旗南農改',
 '東港工作站',
 '林試六龜中心',
 '林試扇平站',
 '林試畢祿溪站',
 '溪口農場',
 '畜試所',
 '種苗繁殖',
 '義竹分場',
 '臺中農改',
 '臺南蘭花園區',
 '臺南農改',
 '臺大內茅埔',
 '臺大和社',
 '臺大溪頭',
 '臺大竹山',
 '臺大雲林校區',
 '臺西水試所',
 '蓮華池',
 '農業試驗所',
 '雲林分場',
 '高雄農改',
 '魚池茶改',
 '鳳山農試']

In [10]:
pd.Timestamp.now().year

2021

In [36]:
def csv_to_df(csv, year):
    
    # 檔案名稱 => 注意斜線與反斜線
    global file_name
    file_name = csv.split("\\")[1].split(".")[0][4:]
    
    # 處理dataframe
    df = pd.read_csv(csv, header=None).T
    df.drop(columns=[0,32], axis=1, inplace=True)
    df.drop(index=0, inplace=True)
    
    # 該年度所有日期
    whole_year_date = pd.date_range(f"{year}-01-01", f"{year}-12-31")
    days = []
    for day in whole_year_date:
        days.append((day.month, day.day))
    
    # 利用日期對應到df的位置
    output = []     
    for d in days:
        month = d[0] - 1
        day = d[1] - 1
        data = df.iloc[month, day]
        
        # 若為數字才append，非數字字元append nan (今年還沒到的日子都被預先設為nan)
        if data != np.nan or re.match('\d', data):
            output.append(data)
        else:
            output.append(np.nan)
    
    # create dataframe
    if year == pd.Timestamp.now().year:
        df_col = pd.DataFrame({"日期": whole_year_date,
                              f"{file_name}": output}).fillna(0)
    else:
        df_col = pd.DataFrame({"日期": whole_year_date,
                              f"{file_name}": output}).fillna(method="ffill")
    
    return df_col


In [43]:
# 打包成自定義涵數
def merged_dataframe_to_csv(target_stations, start_year, end_year):
    # 測站
    for station in target_stations:
        path = f"./cleaned data/Agricultural_Weather_Datas(merged)/{station}"
        if os.path.exists(path) != True:
            os.makedirs(path)
        
        # 年份
        for year in range(start_year, end_year + 1):
            whole_year = pd.date_range(f"{year}-01-01", f"{year}-12-31")
            df_output = pd.DataFrame({"日期": whole_year})
            
            # 合併同年度資料
            for csv in glob.glob(f"../data/Agricultural_Weather_Datas/{station}/{year}*"):
                try:
                    df_col = csv_to_df(csv, year)
                    df_output = pd.merge(df_output, df_col, on="日期", how="left")


                except:
                    print(f"{station}-{year}-{file_name}: Something got wrong!")
                
            # 輸出為csv檔
            df_output.to_csv(f"{path}/{str(year)}_{station}.csv", index=False)
        
            print(f"{station}-{year}: Done" + "="*20)

In [44]:
# 測試
target_stations = ["凍頂茶改"]
start_year = 2019
end_year = 2021

merged_dataframe_to_csv(target_stations, start_year, end_year)

凍頂茶改-2019: Done====================
凍頂茶改-2020: Done====================
凍頂茶改-2021: Done====================
